### Abstract Data Type: The Positional List 
- numeric indicies are not a good abstraction for describing local position in applications where the index of an entry is changing over time due to insertions/deletions happening earlier in the sequence
- for example, a word processor uses the abstraction of a `cursor` to describe a position within the document without explicit use of an integer index, allowing operations such as "delete the character at the cursor" or "insert a new character just after the cursor."